In [ ]:
import os
import re
import pandas as pd 
import openpyxl

def find_name(job_list):
    raw_data = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', "{}.xlsx".format(job_list))
    wb = openpyxl.load_workbook(raw_data) 
    ws = wb.worksheets[0] 

    mr = ws.max_row 

    result = []
    filtrate = re.compile(u'[\u4e00-\u9fff]')
    
    for i in range (2, mr + 1):
        r_list = []
        name = ws.cell(row = i, column = 2)
        info = ws.cell(row = i, column = 3)
        
        p_name = str(name.value).replace('(', '').replace(')', '')
        p_name = filtrate.sub(r'', p_name)
        p_info = str(info.value).split("/")
        
        r_list.append(p_name)
        for i in range(len(p_info)):
            r_list.append(p_info[i])
        
        result.append(r_list)
    
    return result

In [ ]:
def phonor(df):
    h_list = find_name('honor_list')

    H1_AGE, H1_SEX, H2_AGE, H2_SEX, H3_AGE, H3_SEX = list(), list(), list(), list(), list(), list()

    for i in range(len(df)):
        data = df.loc[i, 'honor'] #.split()
        
        try:
            dd = re.findall(r"\d{4}", df.loc[i, 'cid'])[0]
        except:
            dd = '2020'
        
        hn, ha, hs = [str(), str(), str()], [str(), str(), str()], [str(), str(), str()]

        try:
            honors = re.findall('\w+', data)
            h_info = [e for h in honors for e in h_list if h in e]
            for j in range(len(h_info)):
                ha[j] = str((int(dd) - int(h_info[j][1].replace('년생', '')))//10)
                hs[j] = '1' if '여자' in h_info[j][2] else '0' if '남자' in h_info[j][2] else str()
        except:
            pass
        
        H1_AGE.append(ha[0])
        H1_SEX.append(hs[0])
        
        H2_AGE.append(ha[1])
        H2_SEX.append(hs[1])

        H3_AGE.append(ha[2])
        H3_SEX.append(hs[2])
        
    df['h1_age'] = H1_AGE
    df['h1_sex'] = H1_SEX

    df['h2_age'] = H2_AGE
    df['h2_sex'] = H2_SEX

    df['h3_age'] = H3_AGE
    df['h3_sex'] = H3_SEX
    
    return df

In [ ]:
def pprosec(df):
    
    p_list = find_name('prosecutor_list')
    
    P1_AGE, P1_SEX, P2_AGE, P2_SEX = list(), list(), list(), list()

    for i in range(len(df)):
        
        pn, pa, ps = [str(), str()], [str(), str()], [str(), str()]

        try:
            data = df.loc[i, 'prosecutor'].split()
        except:
            P1_AGE.append(pa[0])
            P1_SEX.append(ps[0])

            P2_AGE.append(pa[1])
            P2_SEX.append(ps[1])
            continue
        
        try:
            dd = re.findall(r"\d{4}", df.loc[i, 'cid'])[0]
        except:
            dd = '2020'

        for j in range(len(data)):
            if '기소' in data[j]:
                pn[0] = data[j].replace('기소', '').replace("(", "").replace(")", "").replace("[", "").replace("'", "").replace(",", "")
            elif '공판' in data[j]:
                pn[1] = data[j].replace('공판', '').replace("(", "").replace(")", "").replace("]", "").replace("'", "").replace(",", "")
                
       
        p_info = [e for p in pn for e in p_list if p in e]
        for j in range(len(p_info)):
            if j>1:
                break
            if len(p_info[j])<3:
                continue
                
            try:
                pa[j] = str((int(dd) - int(p_info[j][1].replace('년생', '')))//10)
                ps[j] = '1' if '여자' in p_info[j][2] else '0' if '남자' in p_info[j][2] else str()
            except:
                ps[j] = '1' if '여자' in p_info[j][1] else '0' if '남자' in p_info[j][1] else str()

        P1_AGE.append(pa[0])
        P1_SEX.append(ps[0])
        
        P2_AGE.append(pa[1])
        P2_SEX.append(ps[1])
        
    df['p1_age'] = P1_AGE
    df['p1_sex'] = P1_SEX
    df['p2_age'] = P2_AGE
    df['p2_sex'] = P2_SEX

    return df

In [ ]:
def psentence(df):
    
    SENTENCE, PROBATION, FINE, APPEAL = list(), list(), list(), list()
    count = 0
    
    for i in range(len(df)):
        
        result = [str(), str(), str(), str()]
        try:
            data = df.loc[i, 'sentence'].replace(',', '')
        except:
            pass
        
        try:
            result[0] = re.findall(r'[징역|금고]\s(\d+년)?\s?(\d+?개?월)?', data)[0]
        except:
            pass
        
        if not any(result[0]):
            try:
                result[0] = re.findall(r'[징역|금고]\s(\d+년)\s?(\d+?개?월)?', data)[0]
            except:
                pass
        if not any(result[0]):
            try:
                result[0] = re.findall(r'[장기|단기]\s(\d+년)?\s?(\d+?개?월)?', data)[0]
            except:
                pass
        if not any(result[0]):
            try:
                result[0] = re.findall(r'[장기|단기]\s(\d+년)\s?(\d+?개?월)?', data)[0]
            except:
                pass
        if '무죄' in data:
            result[0] = '0'
        if '무기징역' in data:
            result[0] = '1000'
        if '사형' in data:
            result[0] = '1000'
        if type(result[0]) is tuple:
            if result[0][0] or result[0][1]:
                year = int(re.sub(r'\D', '', result[0][0]))*12  if result[0][0] else 0
                month = int(re.sub(r'\D', '', result[0][1])) if result[0][1] else 0
                result[0] = str(year+month)
            else:
                result[0] = str()
                
        result[1] = '1' if '유예' in data else str()
        
        try:
            result[2] = re.findall(r'벌금\s(\d+만?억?\s?원)', data)[0]
        except:
            pass
        if not result[2]:
            try:
                result[2] = re.findall(r'(\d+만?억?\s?원)', data)[0]
            except:
                pass
        result[2] = str(int(re.sub(r'\D', '', result[2]))*10000) if '만' in result[2] else str(re.sub(r'\D', '', result[2]))
        result[2] = str(int(re.sub(r'\D', '', result[2]))*100000000) if '억' in result[2] else result[2]
        
        result[3] = '1' if '기각' in data else str()
        if result[3] == str():
            result[3] = '0' if '파기' in data else str()
        if result[3] == str():
            result[3] = '0' if '취소' in data else str()
        if result[3] == str():
            result[3] = '0' if '제청' in data else str()
        if result[3] == str():
            result[3] = '0' if '재심을 개시' in data else str()
        
        SENTENCE.append(result[0])
        PROBATION.append(result[1])
        FINE.append(result[2])
        APPEAL.append(result[3])
        
        
        if result == [str(), str(), str(), str()]:
            print(df.loc[i, 'seqnum'], data, "\n\n")
            count += 1
    
    df['y_sentence'] = SENTENCE
    df['y_probation'] = PROBATION
    df['y_fine'] = FINE
    df['y_appeal'] = APPEAL
    print(count)
    
    return df

In [ ]:
def pcase(df):
    CASE = []
    for i in range(len(df)):
        
        data = df.loc[i, 'case']
        try:
            data = re.sub(r'[\(\[].*?[\)\]]', '', data)
        except:
            CASE.append('')
            continue
        data = data.replace('가.', '')
        data = re.sub(r'[^가-힣\, ]', ' ', data)
        data = data.split(',')
        data = [e.strip() for e in data if len(e.strip()) > 1]
        data = [e for e in data if ' ' not in e]
        
        CASE.append(data)
    
    df['case'] = CASE
    return df

In [ ]:
from sklearn.preprocessing import LabelEncoder

xname = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', f"{'xlsx_#5_law'}.xlsx")
df = pd.read_excel(xname, index_col=0)

df1 = pprosec(df)
df2 = phonor(df1)
df3 = psentence(df2)
df4 = pcase(df3)

df4['cbub'] = LabelEncoder().fit_transform(df4['cbub_code'])

df4 = df4[['seqnum', 'cbub', 'title', 'url', 'path', 'text', 'image', 'appeal', 'cid', 'case', 
               'prosecutor', 'p1_age', 'p1_sex', 'p2_age', 'p2_sex', 'honor', 'h1_age', 'h1_sex', 'h2_age', 'h2_sex', 'h3_age', 'h3_sex', 
           'body', 'law', 'sentence', 'y_sentence', 'y_probation', 'y_fine', 'y_appeal']]

for i in range(len(df4)):
    try:
        if df4.loc[i, 'y_sentence'] < '37':
            if df4.loc[i, 'y_appeal'] != '1':
                df4.at[i, 'y_appeal'] = '0'
    except:
        pass
    
df4.reset_index(drop=True, inplace=True)
x2name = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', f"{'xlsx_#6_arrange'}.xlsx")
df4.to_excel(x2name)

#보충작업 필요
#cid, p1_age, p1_sex, h1_age, h1_sex, sentence, case, law
#항소인데 벌금있는 경우 살펴보기

In [ ]:
xname = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', f"{'xlsx_#6_arrange'}.xlsx")
df = pd.read_excel(xname, index_col=0)

df = df.drop(columns=['seqnum', 'path', 'text', 'image', 'prosecutor', 'honor', 'sentence'])
x2name = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', f"{'xlsx_#7_dropped'}.xlsx")
df.to_excel(x2name)